# "Предсказание коэффициента восстановления золота"

Цели проекта  

Подготовить модель машинного обучения для предскаазания коэффициента восстановления золота из золотосодержащей руды. Доступны данные с параметрами добычи и очистки. Модель необходима для оптимизации производства.

Задачи проекта  

Подготовка Данных  
1.1 Изучение данных  
1.2 Проверка расчётов эффективности обогащения  
1.3 Анализ признаков, недоступных в тестовой выборке  
1.4 Предобработка данных  
Анализ данных  
2.1 Анализ изменения концентрации металлов (Au, Ag, Pb)  
2.2 Сравнение распределений размеров гранул сырья  
2.3 Исследование суммарных концентраций веществ  
Построение модели  
3.1 Подготовка метрики итоговой sMAPE  
3.2 Обучение и оценка качества моделей,  
оценка важности признаков методом permutation importance  
3.3 Тестирование модели Lasso  

## Подготовка данных

In [1]:
# библиотеки
import pandas as pd
import numpy as np

Загрузим и рассмотрим набор данных предназначенный для тренировки модели.

In [7]:
train_df =  pd.read_csv(r"C:\Users\Professional\my_project\gold_recovery\gold_recovery_train_new.csv", index_col = 'date')

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 14149 entries, 2016-01-15 00:00:00 to 2018-08-18 10:59:59
Data columns (total 86 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   final.output.concentrate_ag                         14148 non-null  float64
 1   final.output.concentrate_pb                         14148 non-null  float64
 2   final.output.concentrate_sol                        13938 non-null  float64
 3   final.output.concentrate_au                         14149 non-null  float64
 4   final.output.recovery                               14149 non-null  float64
 5   final.output.tail_ag                                14149 non-null  float64
 6   final.output.tail_pb                                14049 non-null  float64
 7   final.output.tail_sol                               14144 non-null  float64
 8   final.output.tail_au                             

<h3><center> Описание данных  </center></h3>

Наименование признаков:  
[этап].[тип_параметра].[название_параметра]  
Пример: rougher.input.feed_ag  
Возможные значения для блока [этап]:  
rougher — флотация  
primary_cleaner — первичная очистка  
secondary_cleaner — вторичная очистка  
final — финальные характеристики  
Возможные значения для блока [тип_параметра]:  
input — параметры сырья  
output — параметры продукта  
state — параметры, характеризующие текущее состояние этапа  
calculation — расчётные характеристики  
Параметры этапов:  
air amount — объём воздуха  
fluid levels — уровень жидкости  
feed size — размер гранул сырья  
feed rate — скорость подачи  

In [10]:
# Установим ограничения в оттображении числа строк и столбцов
pd.set_option('display.max_rows', 87)
pd.set_option('display.max_columns', 50)

In [11]:
#суммарное количество пропусков
print('количество пропущенных значений:', train_df.isnull().sum(), sep='\n')

количество пропущенных значений:
final.output.concentrate_ag                              1
final.output.concentrate_pb                              1
final.output.concentrate_sol                           211
final.output.concentrate_au                              0
final.output.recovery                                    0
final.output.tail_ag                                     0
final.output.tail_pb                                   100
final.output.tail_sol                                    5
final.output.tail_au                                     0
primary_cleaner.input.sulfate                           20
primary_cleaner.input.depressant                        32
primary_cleaner.input.feed_size                          0
primary_cleaner.input.xanthate                         100
primary_cleaner.output.concentrate_ag                    0
primary_cleaner.output.concentrate_pb                   86
primary_cleaner.output.concentrate_sol                 286
primary_cleaner.output.